In [24]:
import shutil
from pathlib import Path
import os
import pandas as pd

In [25]:
!Nvidia-SMI

Thu Feb  1 15:25:37 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8              3W /   65W |    3698MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [26]:
data = pd.read_csv('../Test_Data/summan.smr')

In [27]:
data.head()

,img_name,lat,lon,heading,pitch,timestamp,status,Tree,Pole w/ wire,Intersection
0,_panoid_-1tN4lEA-abrRGZw66d34Q_heading_294_pit...,-23.549251,-46.541349,294,-1,2019-03,free,1,1,0
1,_panoid_-2bCdOFUdXRl3YZkXZ2KVw_heading_224_pit...,-23.524786,-46.695520,224,4,2019-04,free,1,1,1
2,_panoid_-2bCdOFUdXRl3YZkXZ2KVw_heading_334_pit...,-23.524786,-46.695520,334,4,2019-04,free,1,1,1
3,_panoid_-2yEbA_Nn547hSIMJqXFmg_heading_133_pit...,-23.564307,-46.755973,133,6,2019-05,free,1,0,0
4,_panoid_-2yEbA_Nn547hSIMJqXFmg_heading_248_pit...,-23.564307,-46.755973,248,6,2019-05,free,1,0,0


In [28]:
data['img_name'] = dataset_path + data['img_name']
data = data.drop('lat',axis=1)
data = data.drop('lon',axis=1)
data = data.drop('timestamp',axis=1)
data = data.drop('status',axis=1)
data.head()

,img_name,heading,pitch,Tree,Pole w/ wire,Intersection
0,../Test_Data/_panoid_-1tN4lEA-abrRGZw66d34Q_he...,294,-1,1,1,0
1,../Test_Data/_panoid_-2bCdOFUdXRl3YZkXZ2KVw_he...,224,4,1,1,1
2,../Test_Data/_panoid_-2bCdOFUdXRl3YZkXZ2KVw_he...,334,4,1,1,1
3,../Test_Data/_panoid_-2yEbA_Nn547hSIMJqXFmg_he...,133,6,1,0,0
4,../Test_Data/_panoid_-2yEbA_Nn547hSIMJqXFmg_he...,248,6,1,0,0


In [29]:
from PIL import Image
import os

In [30]:
dataset_path = '../Test_Data/'

# Create a list to store file paths and corresponding labels
file_paths = []
labels = []

In [31]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)


In [32]:
train_df.head()

,img_name,heading,pitch,Tree,Pole w/ wire,Intersection
535,../Test_Data/_panoid_-qW3U3yMEE8vz47tLnu1eg_he...,74,0,1,1,1
696,../Test_Data/_panoid_2HlR4Kl9I4I7ZiwABtpsLg_he...,20,0,1,1,0
557,../Test_Data/_panoid_00Y2OYhvDszADp7kTpQPSw_he...,62,-1,1,1,0
643,../Test_Data/_panoid_1oqQcQwNkHIpC62HBv2B3A_he...,200,0,1,1,1
597,../Test_Data/_panoid_0Ral0UQRhTFsLf2RDhJ1RQ_he...,180,-1,1,1,0


In [33]:
train_df.shape,test_df.shape

((800, 6), (201, 6))

In [34]:
import tensorflow as tf

In [35]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_df['img_name'].values, train_df['Intersection'].values)
)
print(train_dataset)

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [36]:
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_df['img_name'].values, test_df['Intersection'].values)
)
print(test_dataset)

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [37]:
def load_and_preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_image(image, channels=3)  # Adjust channels based on your images

    # Explicitly set the shape of the image tensor
    image.set_shape([640, 640, 3])  # Assuming 3 channels for RGB images

    # Resize the image
    image = tf.image.resize(image, (640, 640))  # Adjust size as needed

    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image, label

In [38]:
test_dataset = test_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.map(load_and_preprocess_image)

In [39]:
!Nvidia-SMI

Thu Feb  1 15:25:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8              3W /   65W |    3698MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [40]:
train_dataset = train_dataset.shuffle(buffer_size=len(train_df))
train_dataset = train_dataset.batch(32)

In [41]:
test_dataset = test_dataset.shuffle(buffer_size=len(train_df))
test_dataset = test_dataset.batch(32)

In [42]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(640,640,3),
                                                           include_top=False,
                                                           weights='imagenet')
base_model.trainable = False
from tensorflow.keras import layers, models

# Define your base model
# Build the modified model with an additional convolutional layer
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1000, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
model.summary()
model.load_weights('./MobileNetV2_640_640_100_20finetuned/')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 20, 20, 1280)     2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 1000)              1281000   
                                                                 
 dropout_2 (Dropout)         (None, 1000)              0         
                                                                 
 dense_4 (Dense)             (None, 500)               500500    
                                                                 
 dropout_3 (Dropout)         (None, 500)              

In [43]:
model.evaluate(test_dataset)

7/7 [==============================] - 9s 333ms/step - loss: 0.9475 - accuracy: 0.7164


[0.9475407600402832, 0.7164179086685181]

In [45]:
base_model = tf.keras.applications.DenseNet121(input_shape=(640,640,3),
                                                           include_top=False,
                                                           weights='imagenet')
base_model.trainable = False
from tensorflow.keras import layers, models

# Define your base model
# Build the modified model with an additional convolutional layer
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1000, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
model.summary()
model.load_weights('./Densenet121_640_640_50_20_final/')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 20, 20, 1024)      7037504   
                                                                 
 global_average_pooling2d_3   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_9 (Dense)             (None, 1000)              1025000   
                                                                 
 dropout_6 (Dropout)         (None, 1000)              0         
                                                                 
 dense_10 (Dense)            (None, 500)               500500    
                                                                 
 dropout_7 (Dropout)         (None, 500)               0         
                                                      

In [47]:
model.evaluate(train_dataset)

25/25 [==============================] - 28s 904ms/step - loss: 0.5607 - accuracy: 0.7837


[0.5607309341430664, 0.7837499976158142]

In [48]:
model.evaluate(test_dataset)

7/7 [==============================] - 6s 802ms/step - loss: 0.5992 - accuracy: 0.7114


[0.5992071628570557, 0.711442768573761]

In [59]:
from tensorflow.keras import layers, models
base_model1 = tf.keras.applications.InceptionResNetV2(input_shape=(640,640,3),
                                                           include_top=False,
                                                           weights='imagenet')
base_model1.trainable = False


# Define your base model
# Build the modified model with an additional convolutional layer
model = models.Sequential([
    base_model1,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1000, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
model.summary()
model.load_weights('./InceptionResNetV2_640_640_90/')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 18, 18, 1536)     54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_6   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_18 (Dense)            (None, 1000)              1537000   
                                                                 
 dropout_12 (Dropout)        (None, 1000)              0         
                                                                 
 dense_19 (Dense)            (None, 500)               500500    
                                                                 
 dropout_13 (Dropout)        (None, 500)              

In [60]:
model.evaluate(train_dataset)

25/25 [==============================] - 49s 2s/step - loss: 0.3183 - accuracy: 0.8537


[0.31833669543266296, 0.8537499904632568]

In [61]:
model.evaluate(test_dataset)

7/7 [==============================] - 11s 1s/step - loss: 0.6642 - accuracy: 0.6866


[0.6641801595687866, 0.6865671873092651]

In [63]:
from tensorflow.keras import layers, models
base_model1 = tf.keras.applications.NASNetMobile(input_shape=(640,640,3),
                                                           include_top=False,
                                                           weights='imagenet')
base_model1.trainable = False


# Define your base model
# Build the modified model with an additional convolutional layer
model = models.Sequential([
    base_model1,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1000, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
model.summary()
model.load_weights('./NASNET640_640_640_60NofineTuning/')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 20, 20, 1056)      4269716   
                                                                 
 global_average_pooling2d_8   (None, 1056)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_24 (Dense)            (None, 1000)              1057000   
                                                                 
 dropout_16 (Dropout)        (None, 1000)              0         
                                                                 
 dense_25 (Dense)            (None, 500)               500500    
                                                                 
 dropout_17 (Dropout)        (None, 500)               0         
                                                      

In [72]:
model.evaluate(test_dataset)

7/7 [==============================] - 6s 712ms/step - loss: 0.5413 - accuracy: 0.7114


[0.5412937998771667, 0.711442768573761]

In [73]:
model.evaluate(train_dataset)

25/25 [==============================] - 22s 801ms/step - loss: 0.2616 - accuracy: 0.9125


[0.26155218482017517, 0.9125000238418579]

In [2]:
for i in val_df['img_name']:
    print(i)

NameError: name 'val_df' is not defined